<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
#!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

In [8]:
# Load the CSV file, skipping problematic lines

file_path = '/content/FantasyPros_2024_Overall_ADP_Rankings.csv'
players_df = pd.read_csv(file_path, on_bad_lines='skip')  # or 'warn'

# Select relevant columns
players_df_subset = players_df[['Rank', 'Player', 'Team', 'Bye', 'POS', 'AVG']]

# Compute Value: highest for rank 1, lowest for last rank
max_rank = players_df_subset['Rank'].max()
players_df_subset['Value'] = max_rank - players_df_subset['Rank'] + 1

# Preview the result
print(players_df_subset.head())

   Rank               Player Team Bye  POS  AVG  Value
0   1.0  Christian McCaffrey   SF  9   RB1  1.0  948.0
1   2.0          CeeDee Lamb  DAL   7  WR1  2.6  947.0
2   3.0          Tyreek Hill  MIA   6  WR2  3.2  946.0
3   4.0       Bijan Robinson  ATL  12  RB2  5.0  945.0
4   5.0          Breece Hall  NYJ  12  RB3  5.4  944.0


<ipython-input-8-97671b2087cc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df_subset['Value'] = max_rank - players_df_subset['Rank'] + 1


In [ ]:

# Roster constraints
positions = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'DST': 1}
salary_cap = 50000

# Create model
m = gp.Model("fantasy_draft")

# Decision variables: x[p] = 1 if player p is selected
x = {}
for p in players:
    x[p['name']] = m.addVar(vtype=GRB.BINARY, name=p['name'])

# Objective: maximize total projected points
m.setObjective(gp.quicksum(p['proj_points'] * x[p['name']] for p in players), GRB.MAXIMIZE)

# Position constraints
for pos, count in positions.items():
    m.addConstr(
        gp.quicksum(x[p['name']] for p in players if p['position'] == pos) == count,
        name=f"{pos}_constraint"
    )

# Salary cap constraint
m.addConstr(
    gp.quicksum(p['salary'] * x[p['name']] for p in players) <= salary_cap,
    name="salary_cap"
)

# Solve
m.optimize()

# Output selected players
print("Drafted Players:")
for p in players:
    if x[p['name']].X > 0.5:
        print(f"{p['name']} ({p['position']}) - Projected Points: {p['proj_points']} - Salary: {p['salary']}")
